In [11]:
import av
import numpy as np
import pandas as pd

def sync_video_and_steering(example_segment):
    """
    Sync video frames with steering angle data
    """
    # Load steering data
    steering_times = np.load(example_segment + 'processed_log/CAN/steering_angle/t')
    steering_angles = np.load(example_segment + 'processed_log/CAN/steering_angle/value')
    
    # Load frame times
    frame_times = np.load(example_segment + 'global_pose/frame_times')
    
    print(f"Number of frames: {len(frame_times)}")
    print(f"Number of steering measurements: {len(steering_times)}")
    print(f"\nFrame time range: {frame_times[0]:.2f} to {frame_times[-1]:.2f}")
    print(f"Steering time range: {steering_times[0]:.2f} to {steering_times[-1]:.2f}")
    
    # Create a list to store synchronized data
    synced_data = []
    
    # For each frame time, find the closest steering angle measurement
    for frame_idx, frame_time in enumerate(frame_times):
        # Find the closest steering measurement
        closest_idx = np.argmin(np.abs(steering_times - frame_time))
        
        synced_data.append({
            'frame_idx': frame_idx,
            'frame_time': frame_time,
            'steering_time': steering_times[closest_idx],
            'steering_angle': steering_angles[closest_idx],
            'time_diff': abs(frame_time - steering_times[closest_idx])
        })
    
    # Convert to DataFrame
    df = pd.DataFrame(synced_data)
    
    print("\nSync Statistics:")
    print(f"Average time difference: {df['time_diff'].mean()*1000:.2f} ms")
    print(f"Max time difference: {df['time_diff'].max()*1000:.2f} ms")
    
    # Save synchronized data
    df.to_csv('synced_steering_data.csv', index=False)
    print("\nSynchronized data saved to 'synced_steering_data.csv'")
    
    return df

In [12]:
example_segment = 'data/comma2k19/extracted/Chunk_1/b0c9d2329ad1606b|2018-07-27--06-03-57/3/'
df = sync_video_and_steering(example_segment)
df.to_csv('synced_steering_data.csv', index=False)

Number of frames: 1197
Number of steering measurements: 4973

Frame time range: 9314.09 to 9374.04
Steering time range: 9314.16 to 9374.15

Sync Statistics:
Average time difference: 4.31 ms
Max time difference: 65.34 ms

Synchronized data saved to 'synced_steering_data.csv'


In [13]:
def sync_video_and_steering_with_more_state_data(segment):
    """
    Sync video frames with steering angle data and additional car state parameters
    """
    # Load existing steering data
    steering_times = np.load(segment + 'processed_log/CAN/steering_angle/t')
    steering_angles = np.load(segment + 'processed_log/CAN/steering_angle/value')

    # Load speed data
    speed_times = np.load(segment + 'processed_log/CAN/speed/t')
    speed_values = np.load(segment + 'processed_log/CAN/speed/value')

    # Load IMU data
    gyro_times = np.load(segment + 'processed_log/IMU/gyro/t')
    gyro_values = np.load(segment + 'processed_log/IMU/gyro/value')  # Contains [x,y,z] rotation rates

    # Load frame data
    frame_times = np.load(segment + 'global_pose/frame_times')
    frame_velocities = np.load(segment + 'global_pose/frame_velocities')

    synced_data = []
    
    for frame_idx, frame_time in enumerate(frame_times):
        # Find closest steering measurement
        steer_idx = np.argmin(np.abs(steering_times - frame_time))
        
        # Find closest speed measurement
        speed_idx = np.argmin(np.abs(speed_times - frame_time))
        
        # Find closest gyro measurement
        gyro_idx = np.argmin(np.abs(gyro_times - frame_time))
        
        # Get previous steering angles (as before)
        prev_100ms_threshold = frame_time - 0.10
        prev_100ms_candidates = np.where(steering_times <= prev_100ms_threshold)[0]
        prev_100ms_angle = steering_angles[prev_100ms_candidates[-1]] if len(prev_100ms_candidates) > 0 else 0
        
        prev_200ms_threshold = frame_time - 0.20
        prev_200ms_candidates = np.where(steering_times <= prev_200ms_threshold)[0]
        prev_200ms_angle = steering_angles[prev_200ms_candidates[-1]] if len(prev_200ms_candidates) > 0 else 0

        # Create data point with all synchronized information
        data_point = {
            'frame_idx': frame_idx,
            'frame_time': frame_time,
            
            # Steering data
            'steering_angle': steering_angles[steer_idx],
            'steering_angle_prev_100ms': prev_100ms_angle,
            'steering_angle_prev_200ms': prev_200ms_angle,
            
            # Speed data
            'speed': speed_values[speed_idx],
            
            # IMU data (yaw rate from gyro)
            'yaw_rate': gyro_values[gyro_idx][2],  # z-axis rotation rate
            
            # Vehicle state from frame data
            'velocity_x': frame_velocities[frame_idx][0],  # Forward velocity
            'velocity_y': frame_velocities[frame_idx][1],  # Lateral velocity
            'velocity_z': frame_velocities[frame_idx][2],  # Vertical velocity
            
            # Timing information
            'time_diff_steering': abs(frame_time - steering_times[steer_idx]),
            'time_diff_speed': abs(frame_time - speed_times[speed_idx]),
            'time_diff_gyro': abs(frame_time - gyro_times[gyro_idx])
        }
        
        synced_data.append(data_point)

    # Convert to DataFrame
    df = pd.DataFrame(synced_data)
    
    return df

In [14]:
example_segment = 'data/comma2k19/extracted/Chunk_1/b0c9d2329ad1606b|2018-07-27--06-03-57/3/'
df = sync_video_and_steering_with_more_state_data(example_segment)
# save df to csv
df.to_csv('synced_steering_data_with_more_state_data.csv', index=False)